## Setup

In [1]:
reset -fs

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [3]:
RANDOM_STATE = 28

In [4]:
# Comments File
comments = '../Data/CommentsApril2018.csv'

In [5]:
# Read in File
comm = pd.read_csv(comments)

/home/vietpride12/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Encode newDesk Variable

In [7]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Filter out Unknown & comment commentTypes & Non-NAs
filt = (comm.sectionName != "Unknown") & (comm.commentType == "comment") & comm.sectionName.notna()
comm_filt = comm[filt].copy().reset_index()

In [8]:
# transform and map newDesk categories 
newDesk_le = LabelEncoder()
newDesk_labels = newDesk_le.fit_transform(comm_filt['newDesk'])
comm_filt['newDesk_label'] = newDesk_labels

# encoding newDesk
newDesk_ohe = OneHotEncoder()
newDesk_feature_arr = newDesk_ohe.fit_transform(comm_filt[['newDesk_label']]).toarray()
newDesk_feature_labels = list(newDesk_le.classes_)
newDesk_features = pd.DataFrame(newDesk_feature_arr, columns=newDesk_feature_labels)

## Encode typeOfMaterial Variable

In [9]:
# transform and map typeOfMaterial categories 
material_le = LabelEncoder()
material_labels = material_le.fit_transform(comm_filt['typeOfMaterial'])
comm_filt['material_label'] = material_labels

# encoding typeOfMaterial
material_ohe = OneHotEncoder()
material_feature_arr = material_ohe.fit_transform(comm_filt[['material_label']]).toarray()
material_feature_labels = list(material_le.classes_)
material_features = pd.DataFrame(material_feature_arr, columns=material_feature_labels)

In [10]:
# subsetting comment data frame and concatenating with new features
comm_filt_sub = comm_filt.loc[:,['commentBody', 'sectionName']]
comm_df_ohe = pd.concat([comm_filt_sub, newDesk_features, material_features], axis=1)

In [11]:
comm_df_ohe.head()

,commentBody,sectionName,Arts&Leisure,BookReview,Business,Culture,Dining,Editorial,Express,Foreign,...,Weekend,Well,Editorial,News,News Analysis,Obituary (Obit),Op-Ed,Question,Review,briefing
0,How could the league possibly refuse this offe...,Pro Football,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"So then the execs can be like ""yeah...we will ...",Pro Football,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,I would not want to play chess against these c...,Pro Football,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Could the cheerleaders join the Actors' Equity...,Pro Football,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Iran nuclear deal does nothing but postpon...,Europe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
data = comm_df_ohe.drop('sectionName', axis=1)
sections = comm_df_ohe['sectionName']

In [13]:
train_data, test_data, train_target, test_target = train_test_split(comm_df_ohe, sections, random_state=RANDOM_STATE)

In [14]:
train_feat = train_data.drop('commentBody', axis=1)
train_comm = train_data['commentBody']
test_feat = test_data.drop('commentBody', axis=1)
test_comm = test_data['commentBody']

## Vectorize Comments and Concatenate with other Features

In [15]:
vectorizer = TfidfVectorizer(input='content',
                            analyzer='word',
                            stop_words='english',
                            decode_error='ignore')

In [16]:
vec_train_comm = vectorizer.fit_transform(train_comm).todense()
vec_test_comm = vectorizer.fit_transform(test_comm).todense()

In [17]:
train_data = pd.concat([train_feat, pd.DataFrame(vec_train_comm)], axis=1)
test_data = pd.concat([test_feat, pd.DataFrame(vec_test_comm)], axis=1)

MemoryError: 